<a href="https://colab.research.google.com/github/PieterDujardin/Data-Science--Cheat-Sheet/blob/master/NER/ColabNotebooks/LSTM_CRF_w_embeddings_flair_on_EMEAandMEDLINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# pip uninstall torch


Uninstalling torch-1.5.0+cu101:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.5.0+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
y
  Successfully uninstalled torch-1.5.0+cu101


In [0]:
# pip install torch==1.5.0

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.4.0 which is incompatible.


In [0]:
!pip list

Package                  Version        
------------------------ ---------------
absl-py                  0.9.0          
alabaster                0.7.12         
albumentations           0.1.12         
altair                   4.1.0          
asgiref                  3.2.7          
astor                    0.8.1          
astropy                  4.0.1.post1    
astunparse               1.6.3          
atari-py                 0.2.6          
atomicwrites             1.3.0          
attrs                    19.3.0         
audioread                2.1.8          
autograd                 1.3            
Babel                    2.8.0          
backcall                 0.1.0          
beautifulsoup4           4.6.3          
bleach                   3.1.4          
blis                     0.4.1          
bokeh                    1.4.0          
boto                     2.49.0         
boto3                    1.12.46        
botocore                 1.15.46        
Bottleneck      

In [0]:
!pip install flair

     |████████████████████████████████| 143kB 8.0MB/s 
     |████████████████████████████████| 573kB 15.6MB/s 
     |████████████████████████████████| 256kB 13.7MB/s 
     |████████████████████████████████| 798kB 23.0MB/s 
     |████████████████████████████████| 983kB 33.1MB/s 
     |████████████████████████████████| 1.0MB 52.1MB/s 
     |████████████████████████████████| 890kB 58.7MB/s 
     |████████████████████████████████| 3.7MB 55.3MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=cc9c40c61f978f1d53ec671ca35fd2e242ababd5c0d8a7e47d898071e8286802
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for segtok: filename=segtok-1.5.9-cp36-none-any.whl size=24853 sha256=d3e236930b086ca4f9e7e001d45dfff9955d55dda329d7514d61b1b9cb0d8406
  Stored in directory: /root/.cache/pip/wheels/8b/a3/41/54616e3b92f9a3d1b99fc99955a4089f9e6b1d274e66da250c
  Created wheel for mpld3: fi

In [0]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List
import flair

from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

import torch


In [0]:
# the default behavior is that the model gets put on GPU if available and runs on CPU if there is no GPU.
torch.cuda.is_available()

True

In [0]:
columns = {0: 'text', 1: 'ner'}
data_folder = 'drive/My Drive/Colab Notebooks/#4_BERT_on_EMEA_and_MEDLINE/EMEA_MEDLINE/'

In [0]:
corpus = ColumnCorpus(data_folder, columns,
                              train_file='trainEMEA.txt',
                              test_file='testEMEA.txt',
                              dev_file='validEMEA.txt')


2020-04-30 11:40:15,680 Reading data from drive/My Drive/Colab Notebooks/#4_BERT_on_EMEA_and_MEDLINE/EMEA_MEDLINE
2020-04-30 11:40:15,681 Train: drive/My Drive/Colab Notebooks/#4_BERT_on_EMEA_and_MEDLINE/EMEA_MEDLINE/trainEMEA.txt
2020-04-30 11:40:15,681 Dev: drive/My Drive/Colab Notebooks/#4_BERT_on_EMEA_and_MEDLINE/EMEA_MEDLINE/validEMEA.txt
2020-04-30 11:40:15,682 Test: drive/My Drive/Colab Notebooks/#4_BERT_on_EMEA_and_MEDLINE/EMEA_MEDLINE/testEMEA.txt


In [0]:
len(corpus.train)

1208

In [0]:
# len(corpus.test)+len(corpus.train)+len(corpus.dev)
corpus.dev.sentences[0]

Sentence: "ELANCO Animal Health Teichweg 3 D - 35396 Gießen" - 9 Tokens

In [0]:
tag_type = 'ner'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [0]:
print(tag_dictionary)

Dictionary with 16 tags: <unk>, O, B-MEDI, B-PROB, I-MEDI, B-MEAS, I-PROB, B-PROC, B-ANAT, I-PROC, B-EQUI, I-EQUI, I-ANAT, I-MEAS, <START>, <STOP>


In [0]:
#wordembeddings are the classic embeddings
french_embedding = WordEmbeddings('fr') # fasttextembeddings
# stacked_embeddings = StackedEmbeddings([FlairEmbeddings('fr-forward'), FlairEmbeddings('fr-backward')])

In [0]:
sequencetagger = SequenceTagger(hidden_size=128,
                                        embeddings=french_embedding,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True, dropout=0.5)

In [0]:
trainer = ModelTrainer(sequencetagger, corpus)

In [0]:
# patience is by default 3 
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150, monitor_test=True, embeddings_storage_mode ='gpu', patience=6)

2020-04-30 11:40:29,307 ----------------------------------------------------------------------------------------------------
2020-04-30 11:40:29,309 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (dropout): Dropout(p=0.4, inplace=False)
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, batch_first=True, bidirect

{'dev_loss_history': [tensor(12.1269, device='cuda:0'),
  tensor(9.5444, device='cuda:0'),
  tensor(8.8080, device='cuda:0'),
  tensor(7.5062, device='cuda:0'),
  tensor(6.9256, device='cuda:0'),
  tensor(6.5123, device='cuda:0'),
  tensor(6.3923, device='cuda:0'),
  tensor(6.0213, device='cuda:0'),
  tensor(6.1431, device='cuda:0'),
  tensor(5.3740, device='cuda:0'),
  tensor(5.2737, device='cuda:0'),
  tensor(5.2333, device='cuda:0'),
  tensor(5.8683, device='cuda:0'),
  tensor(4.9671, device='cuda:0'),
  tensor(5.0737, device='cuda:0'),
  tensor(4.6838, device='cuda:0'),
  tensor(4.5959, device='cuda:0'),
  tensor(4.4961, device='cuda:0'),
  tensor(4.4287, device='cuda:0'),
  tensor(4.3543, device='cuda:0'),
  tensor(4.1896, device='cuda:0'),
  tensor(4.2939, device='cuda:0'),
  tensor(4.0903, device='cuda:0'),
  tensor(4.0962, device='cuda:0'),
  tensor(4.0946, device='cuda:0'),
  tensor(4.4441, device='cuda:0'),
  tensor(3.9364, device='cuda:0'),
  tensor(4.0873, device='cuda:0'),

In [0]:
result, _ = sequencetagger.evaluate([corpus.test])

In [0]:
#this is CONll eval style - exact match 
print(result.detailed_results)

In [0]:
# TODO: implement inexact match evaluation - token level

In [0]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

In [0]:
model.tag_type

In [0]:
model.tag_type = 'predicted_ner'

# predict tags
pred = model.predict(corpus.test)

In [0]:
pred[36].to_tagged_string('predicted_ner')

In [0]:
pred[36].to_tagged_string('ner')